In [1]:
from keras.models import load_model
from os import listdir
from scipy import misc
from skimage import io,color
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import _pickle as cPickle
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,Sequential
from keras import optimizers
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Conv2D,MaxPooling2D,Activation,Flatten,BatchNormalization
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from skimage import io,color
from scipy import misc
import matplotlib.pyplot as plt
import os.path
import numpy as np
import pandas as pd


%matplotlib inline

/usr/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
print("Loading model")
model=load_model("Model6.h5")
model.pop()
model.pop()
model.pop()
# model.pop()
# model.pop()
print(model.summary())

Loading model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496  

In [3]:
def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model = InceptionV3(weights=weights, include_top=False)
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer
    x = Dropout(0.5)(x)
    predictions = Dense(9, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [3]:
categories=["Golf","Kicking","Lifting","Riding","Running","SkateBoarding","Swing-Bench","Swing-SideAngle","Walking"]
path="Train/"
rnn_train_x=[]
rnn_test_x=[]
rnn_train_y=[]
rnn_test_y=[]
for c in categories:
    items=listdir(path+c)
    for i in items:
        train_images=[]
        print(c+" "+i)
        images=listdir(path+c+'/'+i+'/')
        for imgs in range(len(images)):
#             print(imgs)
            img=io.imread(path+c+'/'+i+'/'+str(imgs)+'.jpg')
            img=misc.imresize(img,(64,64,3))
            img=img*(1./255)
            if len(train_images)==0:
                train_images.append(img)
            
            else:
                train_images.append(img)
            
        x_train=np.array(train_images)
        x_predict=model.predict(x_train) 
#         print(x_predict.shape)
        if x_predict.shape[0]<50:
            x_predict=np.vstack((x_predict,np.zeros((50-x_predict.shape[0],x_predict.shape[1]))))
        else:
            jump=int(x_predict.shape[0]/50)
            x_train_new=[]
            for i in range(0,x_predict.shape[0],jump)[:50]:
                if i==0:
                    x_train_new=(x_predict[i])
                else:
                    x_train_new=np.vstack((x_train_new,x_predict[i]))
            x_predict=np.array(x_train_new)
#         print(x_predict.shape)
        
        y_onehot=np.zeros([1,9])
        y_onehot[0,categories.index(c)]=1
        
        if len(rnn_train_x)==0:
            rnn_train_x.append(x_predict)
            rnn_train_y=y_onehot
        else:
            rnn_train_x.append(x_predict)
            rnn_train_y=np.vstack((rnn_train_y,y_onehot))

path1='Test/'
for c in categories:
    items=listdir(path1+c)
    for i in items:
        train_images=[]
        print(c+" "+i)
        images=listdir(path1+c+'/'+i+'/')
        for imgs in range(len(images)):
#             print(imgs)
            img=io.imread(path1+c+'/'+i+'/'+str(imgs)+'.jpg')
            img=misc.imresize(img,(64,64,3))
            img=img*(1./255)
            if len(train_images)==0:
                train_images.append(img)
            
            else:
                train_images.append(img)
            
        x_train=np.array(train_images)
        x_predict=model.predict(x_train) 
#         print(x_predict.shape)
        if x_predict.shape[0]<50:
            x_predict=np.vstack((x_predict,np.zeros((50-x_predict.shape[0],x_predict.shape[1]))))
        else:
            jump=int(x_predict.shape[0]/50)
            x_train_new=[]
            for i in range(0,x_predict.shape[0],jump)[:50]:
                if i==0:
                    x_train_new=(x_predict[i])
                else:
                    x_train_new=np.vstack((x_train_new,x_predict[i]))
            x_predict=np.array(x_train_new)
#         print(x_predict.shape)
        
        y_onehot=np.zeros([1,9])
        y_onehot[0,categories.index(c)]=1
        
        if len(rnn_test_x)==0:
            rnn_test_x.append(x_predict)
            rnn_test_y=y_onehot
        else:
            rnn_test_x.append(x_predict)
            rnn_test_y=np.vstack((rnn_test_y,y_onehot))
        
rnn_train_x=np.array(rnn_train_x)
rnn_test_x=np.array(rnn_test_x)
train_data=list(zip(rnn_train_x,rnn_train_y))
test_data=list(zip(rnn_test_x,rnn_test_y))

cPickle.dump(train_data,open('RNN_Train_6.p','wb'))
cPickle.dump(test_data,open('RNN_Test_6.p','wb'))
# print(rnn_train_x.shape)
# print(rnn_train_y.shape)
            

Golf 0


/home/kathan/.local/lib/python3.4/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


Golf 1
Golf 10
Golf 12
Golf 13
Golf 2
Golf 5
Golf 6
Golf 7
Golf 8
Golf 9
Kicking 0
Kicking 10
Kicking 11
Kicking 12
Kicking 15
Kicking 2
Kicking 3
Kicking 4
Kicking 6
Kicking 7
Kicking 8
Kicking 9
Lifting 0
Lifting 1
Lifting 3
Lifting 4
Riding 0
Riding 1
Riding 3
Riding 4
Riding 6
Riding 7
Riding 8
Riding 9
Running 0
Running 10
Running 2
Running 3
Running 4
Running 5
Running 6
Running 7
SkateBoarding 0
SkateBoarding 1
SkateBoarding 2
SkateBoarding 3
SkateBoarding 4
SkateBoarding 5
SkateBoarding 8
SkateBoarding 9
Swing-Bench 1
Swing-Bench 10
Swing-Bench 11
Swing-Bench 12
Swing-Bench 13
Swing-Bench 14
Swing-Bench 15
Swing-Bench 2
Swing-Bench 4
Swing-Bench 6
Swing-Bench 7
Swing-Bench 8
Swing-Bench 9
Swing-SideAngle 1
Swing-SideAngle 10
Swing-SideAngle 2
Swing-SideAngle 3
Swing-SideAngle 5
Swing-SideAngle 6
Swing-SideAngle 7
Swing-SideAngle 9
Walking 0
Walking 1
Walking 10
Walking 11
Walking 12
Walking 13
Walking 14
Walking 16
Walking 17
Walking 2
Walking 3
Walking 4
Walking 6
Walking 8
Wa

/home/kathan/.local/lib/python3.4/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Golf 3
Golf 4
Kicking 1
Kicking 13
Kicking 14
Kicking 5
Lifting 2
Riding 2
Riding 5
Running 1
Running 8
Running 9
SkateBoarding 6
SkateBoarding 7
Swing-Bench 0
Swing-Bench 16
Swing-Bench 3
Swing-Bench 5
Swing-SideAngle 0
Swing-SideAngle 4
Swing-SideAngle 8
Walking 15
Walking 18
Walking 5
Walking 7


In [3]:
model.save('Cut_Model2.h5')

In [4]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
dropout_33 (Dropout)         (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 26, 26, 128)       147584    
_________________________________________________________________
dropout_34 (Dropout)         (None, 26, 26, 128)       0         
__________